In [52]:
import torch
from torchvision import transforms
from PIL import Image
import pandas as pd
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

class PretrainedEfficientNet(nn.Module):
    def __init__(self, num_class=7):  # Adjust `num_class` based on your dataset
        super(PretrainedEfficientNet, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')  # Use the correct EfficientNet version
        self.fc1 = nn.Linear(self.efficientnet._fc.in_features, num_class)

    def forward(self, x):
        x = self.efficientnet.extract_features(x)
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


# Load the trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "best_model.pth"
model = PretrainedEfficientNet(num_class=7)  # Replace with your model class
model.load_state_dict(torch.load(model_path, map_location=device,weights_only=True))
model.eval()
model.to(device)

# Load the metadata CSV
metadata_path ="HAM10000_metadata.csv"
metadata = pd.read_csv(metadata_path)

# Define image transformations
input_size = 224  # Replace with your model's input size
norm_mean = [0.7630362, 0.5456468, 0.5700442]  # Replace with dataset's mean
norm_std = [0.14092818, 0.15261286, 0.16997077]  # Replace with dataset's std
transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

# Lesion type dictionary (use the mapping in your dataset)
lesion_type_dict = {
    'akiec': 'Actinic keratoses',
    'bcc': 'Basal cell carcinoma',
    'bkl': 'Benign keratosis-like lesions ',    
    'df': 'Dermatofibroma',
    'nv': 'Melanocytic nevi',
    'vasc': 'Vascular lesions',
    'mel': 'melanoma'
}

def predict_image(image_path):
    """Predicts the class of an input image and compares it with the actual value."""
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Predict
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted_class = torch.max(output, 1)
    
    # Map prediction to class name
    predicted_class_idx = predicted_class.item()
    predicted_class_name = list(lesion_type_dict.values())[predicted_class_idx]
    
    # Find actual class from metadata
    image_id = image_path.split('/')[-1].split('.')[0]
    actual_row = metadata[metadata['image_id'] == image_id]
    
    if not actual_row.empty:
        actual_class_code = actual_row['dx'].values[0]
        actual_class_name = lesion_type_dict[actual_class_code]
        print(f"Predicted: {predicted_class_name}, Actual: {actual_class_name}")
    else:
        print(f"Image ID {image_id} not found in metadata.")
    
    return predicted_class_name

# Example usage
image_name = input("Enter the Image name")
image_path = f"D:\Coding playground\Major Project\data_set\HAM10000_images/{image_name}.jpg"  # Replace with your image path
predict_image(image_path)

Loaded pretrained weights for efficientnet-b0


Enter the Image name ISIC_0025450


Predicted: melanoma, Actual: melanoma


'melanoma'

In [48]:
print("Label to index mapping:")
print({v: k for k, v in enumerate(lesion_type_dict.values())})


Label to index mapping:
{'Actinic keratoses': 0, 'Basal cell carcinoma': 1, 'Benign keratosis-like lesions ': 2, 'Dermatofibroma': 3, 'Melanocytic nevi': 4, 'Vascular lesions': 5, 'melanoma': 6}


In [54]:
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.softmax(output, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1)
    
    predicted_class_idx = predicted_class.item()
    predicted_class_name = list(lesion_type_dict.values())[predicted_class_idx]
    
    # Find actual class
    image_id = image_path.split('/')[-1].split('.')[0]
    actual_row = metadata[metadata['image_id'] == image_id]
    if not actual_row.empty:
        actual_class_code = actual_row['dx'].values[0]
        actual_class_name = lesion_type_dict[actual_class_code]
    else:
        actual_class_name = "Unknown"
    
    print(f"Predicted Class: {predicted_class_name} ({predicted_class_idx})")
    print(f"Actual Class: {actual_class_name}")
    print(f"Prediction Probabilities: {probabilities.cpu().numpy()}")
    return predicted_class_name
predict_image('D:\Coding playground\Major Project\data_set\HAM10000_images/ISIC_0025790.jpg')

Predicted Class: Actinic keratoses (0)
Actual Class: Actinic keratoses
Prediction Probabilities: [[9.9998558e-01 7.5395013e-07 1.3354735e-05 4.2877719e-08 3.3035917e-07
  4.1658923e-11 2.0714626e-08]]


'Actinic keratoses'

In [60]:
# another try of prediction demo 